In [1]:
import pandas as pd
import numpy as np
import math
import re

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold , GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgbs
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
haz_train=pd.read_csv(r'C:\Users\Priyanshi\Downloads\python\Hazard_train.csv')
haz_test=pd.read_csv(r'C:\Users\Priyanshi\Downloads\python\Hazard_test_share.csv')

In [3]:
haz_train.head(4)

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
0,83642,2,10,23,4,N,K,Y,B,D,...,2,37,1,11,2,Y,N,E,2,1
1,92530,6,3,18,2,B,H,N,B,B,...,2,34,1,12,7,Y,N,C,3,3
2,86131,4,5,20,4,B,K,N,B,B,...,1,40,1,17,4,Y,N,D,1,1
3,88666,1,4,7,5,N,H,N,B,B,...,2,37,1,11,6,Y,N,D,2,1


In [4]:
haz_train.shape,haz_test.shape

((40799, 34), (10200, 33))

In [5]:
haz_train.dtypes

Id         int64
Hazard     int64
T1_V1      int64
T1_V2      int64
T1_V3      int64
T1_V4     object
T1_V5     object
T1_V6     object
T1_V7     object
T1_V8     object
T1_V9     object
T1_V10     int64
T1_V11    object
T1_V12    object
T1_V13     int64
T1_V14     int64
T1_V15    object
T1_V16    object
T1_V17    object
T2_V1      int64
T2_V2      int64
T2_V3     object
T2_V4      int64
T2_V5     object
T2_V6      int64
T2_V7      int64
T2_V8      int64
T2_V9      int64
T2_V10     int64
T2_V11    object
T2_V12    object
T2_V13    object
T2_V14     int64
T2_V15     int64
dtype: object

In [6]:
haz_train.select_dtypes(object).nunique().sort_values()

T1_V6      2
T1_V17     2
T2_V3      2
T2_V11     2
T2_V12     2
T1_V7      4
T1_V8      4
T1_V12     4
T2_V13     5
T1_V9      6
T2_V5      6
T1_V4      8
T1_V15     8
T1_V5     10
T1_V11    12
T1_V16    18
dtype: int64

In [7]:
haz_train["Hazard"].value_counts()

1     15229
4      6561
2      4939
5      2902
7      2651
3      1632
8      1434
6      1114
10     1048
11      633
9       594
13      442
12      311
14      304
16      183
17      154
15      142
19      111
18       90
20       71
22       53
25       34
23       31
28       27
21       20
26       20
29       12
24       11
34        6
31        5
30        5
27        4
32        4
38        2
37        2
41        2
40        2
42        2
46        2
64        1
33        1
63        1
35        1
69        1
39        1
49        1
51        1
52        1
44        1
Name: Hazard, dtype: int64

In [8]:
haz_train.isnull().sum() #no null values

Id        0
Hazard    0
T1_V1     0
T1_V2     0
T1_V3     0
T1_V4     0
T1_V5     0
T1_V6     0
T1_V7     0
T1_V8     0
T1_V9     0
T1_V10    0
T1_V11    0
T1_V12    0
T1_V13    0
T1_V14    0
T1_V15    0
T1_V16    0
T1_V17    0
T2_V1     0
T2_V2     0
T2_V3     0
T2_V4     0
T2_V5     0
T2_V6     0
T2_V7     0
T2_V8     0
T2_V9     0
T2_V10    0
T2_V11    0
T2_V12    0
T2_V13    0
T2_V14    0
T2_V15    0
dtype: int64

In [9]:
round(haz_train.groupby("T1_V4")["Hazard"].mean(),2)

T1_V4
B    3.63
C    3.12
E    3.78
G    4.25
H    4.63
N    4.34
S    3.87
W    4.36
Name: Hazard, dtype: float64

In [10]:
haz_train.groupby("T2_V13")["Hazard"].median().sort_values()

T2_V13
A    2
E    2
C    3
D    3
B    4
Name: Hazard, dtype: int64

In [11]:
haz_train["T2_V13"].value_counts()

E    22008
A     8215
C     6032
D     4126
B      418
Name: T2_V13, dtype: int64

In [12]:
haz_train["T1_V6"].value_counts()

N    22856
Y    17943
Name: T1_V6, dtype: int64

In [13]:
haz_train.nunique().sort_values()

T2_V12        2
T2_V11        2
T1_V17        2
T1_V6         2
T2_V3         2
T2_V8         3
T1_V7         4
T1_V8         4
T1_V13        4
T1_V12        4
T1_V10        5
T2_V13        5
T1_V14        5
T2_V5         6
T1_V9         6
T2_V14        7
T2_V7         7
T2_V6         7
T2_V10        7
T1_V4         8
T1_V15        8
T1_V3         9
T1_V5        10
T2_V15       12
T1_V11       12
T1_V16       18
T1_V1        19
T2_V4        22
T1_V2        24
T2_V9        25
T2_V2        39
Hazard       49
T2_V1       100
Id        40799
dtype: int64

In [14]:
haz_train["T1_V11"].value_counts()

B    13694
H    12313
L     5590
J     4961
A     1257
I     1073
F      427
M      417
E      352
N      333
D      200
K      182
Name: T1_V11, dtype: int64

In [15]:
haz_train.describe()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V10,T1_V13,T1_V14,T2_V1,T2_V2,T2_V4,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V14,T2_V15
count,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000,40799.000000
mean,50850.454276,4.009584,9.723645,12.849359,3.190691,7.025123,14.009902,1.582416,57.568641,12.410451,10.251403,1.947499,33.504620,1.031349,12.496973,4.489963,2.448344,3.482512
std,29282.641037,4.007785,5.170715,6.257502,1.742044,3.596619,4.649012,0.865121,23.461468,4.785406,4.845443,0.795249,5.830512,0.191676,7.313552,1.899386,1.257293,3.070911
min,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,5.000000,0.000000,1.000000,1.000000,1.000000,1.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,25596.500000,1.000000,6.000000,7.000000,2.000000,3.000000,10.000000,1.000000,40.000000,9.000000,6.000000,2.000000,31.000000,1.000000,6.000000,3.000000,2.000000,1.000000
50%,50784.000000,3.000000,9.000000,14.000000,3.000000,8.000000,15.000000,1.000000,55.000000,11.000000,10.000000,2.000000,34.000000,1.000000,14.000000,4.000000,2.000000,2.000000
75%,76198.500000,5.000000,14.000000,18.000000,4.000000,12.000000,20.000000,2.000000,77.000000,15.000000,14.000000,2.000000,40.000000,1.000000,18.000000,6.000000,3.000000,5.000000
max,101999.000000,69.000000,19.000000,24.000000,9.000000,12.000000,20.000000,4.000000,100.000000,39.000000,22.000000,7.000000,40.000000,3.000000,25.000000,7.000000,7.000000,12.000000


In [16]:
x_train,x_test=train_test_split(haz_train,test_size=.2,random_state=1)

In [17]:
x_train1=x_train.drop(["Hazard","Id"],1)
y_train1=x_train["Hazard"]
x_test1=x_test.drop(["Hazard","Id"],1)
y_test1=x_test["Hazard"]

In [18]:
x_train1.shape,y_train1.shape

((32639, 32), (32639,))

In [19]:
x_test1.shape,y_test1.shape

((8160, 32), (8160,))

In [20]:
num_col=x_train1.select_dtypes(np.number).columns

In [21]:
char_col=x_train1.select_dtypes(object).columns

In [22]:
pipe_num=make_pipeline(SimpleImputer(strategy="median"),StandardScaler())
pipe_col=make_pipeline(OneHotEncoder(handle_unknown="ignore"))

In [23]:
ctrans=make_column_transformer((pipe_num,num_col),(pipe_col,char_col))

In [24]:
ctrans.fit_transform(x_train1)

<32639x111 sparse matrix of type '<class 'numpy.float64'>'
	with 1044448 stored elements in Compressed Sparse Row format>

In [25]:
ctrans.transform(haz_test)

C:\Users\Priyanshi\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


<10200x111 sparse matrix of type '<class 'numpy.float64'>'
	with 326400 stored elements in Compressed Sparse Row format>

## Linear Regression

In [30]:
lm=LinearRegression()

In [31]:
pipe=make_pipeline(ctrans,lm)

In [32]:
pipe

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

In [33]:
pipe.fit(x_train1,y_train1)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

In [34]:
pred_train=pipe.predict(x_train1)
pred_test=pipe.predict(x_test1)

In [35]:
pred_test,pred_train

(array([5.06784005, 2.8998346 , 3.34962519, ..., 7.58674443, 4.27935508,
        2.3990444 ]),
 array([6.23704286, 3.70555443, 3.64790938, ..., 3.52620713, 1.020261  ,
        4.37907751]))

In [36]:
mean_absolute_error(y_test1,pred_test)

2.730216927394613

In [37]:
1-(mean_absolute_error(y_test1,pred_test)/5.4)

0.49440427270470133

In [38]:
pipe.predict(haz_test)

C:\Users\Priyanshi\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


array([7.68125889, 4.05087223, 7.30801925, ..., 4.05894629, 2.63713573,
       2.88406151])

## GridSearchCV

In [39]:
lasso_params = {'classifier__alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
# ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

In [40]:
lasso=Lasso()

In [41]:
#dir(lasso)

In [42]:
model_pipeline=Pipeline([("Preprocessor",ctrans),("classifier",Lasso())])
#model_pipeline=make_pipeline(ctrans,lasso)

In [43]:
model_pipeline

Pipeline(memory=None,
         steps=[('Preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                

In [44]:
grid=GridSearchCV(model_pipeline,param_grid=lasso_params,cv=5)

In [45]:
grid

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('Preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                       

In [46]:
grid.fit(x_train1,y_train1)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('Preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                       

In [47]:
grid.predict(x_train1)

array([5.79073755, 3.72365219, 3.68756212, ..., 3.51095921, 2.21438397,
       4.23726832])

In [48]:
mean_absolute_error(y_train1,grid.predict(x_train1))

2.755582671063343

In [49]:
mae=mean_absolute_error(y_test1,grid.predict(x_test1))

In [50]:
1-(mae/5.4)

0.4925798219575297

## Random Forest

In [51]:
strings='randomforestregressor__'

param_dict={'n_estimators':[int(x) for x in np.linspace(200,1000,num=10)],
           'max_features':['auto','sqrt'],
            'max_depth':[int(x) for x in np.linspace(10,110,num=11)],
            'min_samples_split':[10,20,25],
            'min_samples_leaf':[4,8,10],
            'bootstrap':[True,False]
            }

In [52]:
param_dict.items()

dict_items([('n_estimators', [200, 288, 377, 466, 555, 644, 733, 822, 911, 1000]), ('max_features', ['auto', 'sqrt']), ('max_depth', [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]), ('min_samples_split', [10, 20, 25]), ('min_samples_leaf', [4, 8, 10]), ('bootstrap', [True, False])])

In [53]:
param_dict={strings+k:v for k,v in param_dict.items()}

In [54]:
param_dict

{'randomforestregressor__n_estimators': [200,
  288,
  377,
  466,
  555,
  644,
  733,
  822,
  911,
  1000],
 'randomforestregressor__max_features': ['auto', 'sqrt'],
 'randomforestregressor__max_depth': [10,
  20,
  30,
  40,
  50,
  60,
  70,
  80,
  90,
  100,
  110],
 'randomforestregressor__min_samples_split': [10, 20, 25],
 'randomforestregressor__min_samples_leaf': [4, 8, 10],
 'randomforestregressor__bootstrap': [True, False]}

In [55]:
ctrans.fit_transform(x_train1)

<32639x111 sparse matrix of type '<class 'numpy.float64'>'
	with 1044448 stored elements in Compressed Sparse Row format>

In [56]:
rf=RandomForestRegressor(random_state=1)
model_pipeline=make_pipeline(ctrans,rf)

In [57]:
model_pipeline

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

In [58]:
grid=RandomizedSearchCV(model_pipeline,param_dict,cv=5,scoring="neg_mean_absolute_error")

In [59]:
grid

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('columntransformer',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('simpleimputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                                   

In [ ]:
grid.fit(x_train1,y_train1)

In [54]:
grid.predict(x_train1)

array([4.7996126 , 3.89878352, 2.68234137, ..., 3.50719185, 2.21578207,
       6.42729997])

In [55]:
grid.predict(x_test1)

array([5.82006073, 3.27441548, 3.47022024, ..., 5.18448114, 4.50238091,
       2.77292822])

In [63]:
grid.best_estimator_['randomforestregressor'].feature_importances_[0:5]

array([0.05921782, 0.06408086, 0.03662228, 0.02703719, 0.02193069])

In [70]:
final_pred=grid.predict(haz_test)

C:\Users\Priyanshi\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [71]:
submission_1=pd.DataFrame(data=final_pred)

In [73]:
submission_1.columns=["Hazard"]

In [78]:
submission_1.head()

,Id,Hazard
0,67943,5.486684
1,38287,3.659544
2,51951,6.460168
3,40476,5.124472
4,101190,3.481345


In [75]:
submission_1["Id"]=haz_test.Id

In [77]:
submission_1=submission_1[["Id","Hazard"]]

In [79]:
submission_1=submission_1.to_csv("Project_4_Priyanshi.csv",index=False)

In [56]:
mae=mean_absolute_error(y_test1,grid.predict(x_test1))

In [57]:
mae

2.7038578672624802

In [58]:
1-(mae/5.4)

0.49928558013657776

In [67]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__n_estimators,param_randomforestregressor__min_samples_split,param_randomforestregressor__min_samples_leaf,param_randomforestregressor__max_features,param_randomforestregressor__max_depth,param_randomforestregressor__bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,285.833837,4.667791,1.131156,0.024353,555,10,4,sqrt,110,False,"{'randomforestregressor__n_estimators': 555, '...",-2.671823,-2.708517,-2.746805,-2.743865,-2.753052,-2.724812,0.030717,1
1,80.578103,0.119772,0.393664,0.028285,200,10,8,sqrt,80,False,"{'randomforestregressor__n_estimators': 200, '...",-2.675165,-2.712810,-2.749732,-2.747221,-2.754560,-2.727898,0.030230,5
2,73.861210,0.259979,0.373249,0.014473,200,25,8,sqrt,30,False,"{'randomforestregressor__n_estimators': 200, '...",-2.675304,-2.714860,-2.747373,-2.749107,-2.755945,-2.728518,0.030166,6
3,3793.565921,314.162196,4.040946,2.667699,1000,10,8,auto,50,False,"{'randomforestregressor__n_estimators': 1000, ...",-3.125772,-3.106873,-3.147002,-3.126097,-3.175337,-3.136216,0.023320,10
4,643.898562,40.281224,0.475118,0.027590,288,25,8,auto,20,True,"{'randomforestregressor__n_estimators': 288, '...",-2.674869,-2.711375,-2.749471,-2.742782,-2.760022,-2.727704,0.030996,4
5,255.759911,5.299603,1.364913,0.102716,733,20,4,sqrt,60,True,"{'randomforestregressor__n_estimators': 733, '...",-2.683044,-2.717695,-2.749998,-2.753076,-2.761028,-2.732968,0.029008,7
6,325.464919,10.027631,1.277667,0.051155,555,10,4,sqrt,70,False,"{'randomforestregressor__n_estimators': 555, '...",-2.671823,-2.708517,-2.746805,-2.743865,-2.753052,-2.724812,0.030717,1
7,1478.309168,40.379148,0.753790,0.055233,377,20,8,auto,90,False,"{'randomforestregressor__n_estimators': 377, '...",-3.082352,-3.072892,-3.114369,-3.093913,-3.140885,-3.100882,0.024321,9
8,700.967723,26.935591,0.502938,0.022133,288,25,10,auto,40,True,"{'randomforestregressor__n_estimators': 288, '...",-2.673805,-2.710059,-2.746298,-2.741731,-2.758297,-2.726038,0.030598,3
9,296.061844,7.174529,0.378491,0.019991,288,20,4,auto,10,False,"{'randomforestregressor__n_estimators': 288, '...",-2.794822,-2.806677,-2.851302,-2.860253,-2.869856,-2.836582,0.030074,8


In [65]:
grid.predict(haz_test)

C:\Users\Priyanshi\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

## XgBoost

In [26]:
strings="xgb__"
parameters = {
    'max_depth':range(2,10,1),
    'n_estimators': [40,60,100,150],
    'learning_rate': [0.1, 0.01,0.05], #shrinkage.. reduces overfitting
    'reg_lambda':[0.1,0.01,1,10],
    'reg_alpha':[0.1,0.01,1,10],
}

In [27]:
parameters.items()

dict_items([('max_depth', range(2, 10)), ('n_estimators', [40, 60, 100, 150]), ('learning_rate', [0.1, 0.01, 0.05]), ('reg_lambda', [0.1, 0.01, 1, 10]), ('reg_alpha', [0.1, 0.01, 1, 10])])

In [28]:
xgb_params={strings+k:v for k,v in parameters.items()}

In [29]:
xgb_params

{'xgb__max_depth': range(2, 10),
 'xgb__n_estimators': [40, 60, 100, 150],
 'xgb__learning_rate': [0.1, 0.01, 0.05],
 'xgb__reg_lambda': [0.1, 0.01, 1, 10],
 'xgb__reg_alpha': [0.1, 0.01, 1, 10]}

In [30]:
xgb=xgbs.XGBRegressor()
pipe=Pipeline([("columntransfer",ctrans),("xgb",xgb)])

In [31]:
grid=RandomizedSearchCV(pipe,xgb_params,cv=5,scoring="neg_mean_absolute_error")

In [32]:
grid

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('columntransfer',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('simpleimputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                                      

In [33]:
grid.fit(x_train1,y_train1);

[17:46:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:48:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:48:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:48:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [34]:
grid.predict(x_train1)

array([3.261425 , 2.2580113, 2.2767527, ..., 2.2769709, 1.6621374,
       3.2389414], dtype=float32)

In [35]:
mae=mean_absolute_error(y_test1,grid.predict(x_test1))

In [36]:
mae

2.6170501518337166

In [37]:
1-(mae/5.4)

0.5153610829937563

In [38]:
final_pred=grid.predict(haz_test)

C:\Users\Priyanshi\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [39]:
final_pred

array([3.9606767, 2.3539276, 4.691771 , ..., 2.6365213, 2.314605 ,
       1.8699675], dtype=float32)

In [40]:
submission=pd.DataFrame(data=final_pred)

In [41]:
submission.astype(int).head()

,0
0,3
1,2
2,4
3,4
4,2


In [42]:
submission.head()

,0
0,3.960677
1,2.353928
2,4.691771
3,4.835200
4,2.230071


In [43]:
submission.columns=["Hazard"]

In [44]:
submission.shape

(10200, 1)

In [45]:
haz_test.shape

(10200, 33)

In [46]:
submission=submission.to_csv("Priyanshi_Project4.csv",index=False)

## feature importance

In [47]:
onehotcols =  ctrans.named_transformers_['pipeline-2']['onehotencoder'].get_feature_names()

In [48]:
onehotcols

array(['x0_B', 'x0_C', 'x0_E', 'x0_G', 'x0_H', 'x0_N', 'x0_S', 'x0_W',
       'x1_A', 'x1_B', 'x1_C', 'x1_D', 'x1_E', 'x1_H', 'x1_I', 'x1_J',
       'x1_K', 'x1_L', 'x2_N', 'x2_Y', 'x3_A', 'x3_B', 'x3_C', 'x3_D',
       'x4_A', 'x4_B', 'x4_C', 'x4_D', 'x5_B', 'x5_C', 'x5_D', 'x5_E',
       'x5_F', 'x5_G', 'x6_A', 'x6_B', 'x6_D', 'x6_E', 'x6_F', 'x6_H',
       'x6_I', 'x6_J', 'x6_K', 'x6_L', 'x6_M', 'x6_N', 'x7_A', 'x7_B',
       'x7_C', 'x7_D', 'x8_A', 'x8_C', 'x8_D', 'x8_F', 'x8_H', 'x8_N',
       'x8_S', 'x8_W', 'x9_A', 'x9_B', 'x9_C', 'x9_D', 'x9_E', 'x9_F',
       'x9_G', 'x9_H', 'x9_I', 'x9_J', 'x9_K', 'x9_L', 'x9_M', 'x9_N',
       'x9_O', 'x9_P', 'x9_Q', 'x9_R', 'x10_N', 'x10_Y', 'x11_N', 'x11_Y',
       'x12_A', 'x12_B', 'x12_C', 'x12_D', 'x12_E', 'x12_F', 'x13_N',
       'x13_Y', 'x14_N', 'x14_Y', 'x15_A', 'x15_B', 'x15_C', 'x15_D',
       'x15_E'], dtype=object)

In [49]:
num_col

Index(['T1_V1', 'T1_V2', 'T1_V3', 'T1_V10', 'T1_V13', 'T1_V14', 'T2_V1',
       'T2_V2', 'T2_V4', 'T2_V6', 'T2_V7', 'T2_V8', 'T2_V9', 'T2_V10',
       'T2_V14', 'T2_V15'],
      dtype='object')

In [50]:
all_col=list(num_col)+ list(onehotcols)

In [51]:
grid.best_params_

{'xgb__reg_lambda': 1,
 'xgb__reg_alpha': 1,
 'xgb__n_estimators': 100,
 'xgb__max_depth': 9,
 'xgb__learning_rate': 0.01}

In [54]:
pd.DataFrame(grid.best_estimator_["xgb"].feature_importances_,index=all_col).reset_index(drop=False)

,index,0
0,T1_V1,0.005752
1,T1_V2,0.007339
2,T1_V3,0.004562
3,T1_V10,0.002753
4,T1_V13,0.002825
...,...,...
106,x15_A,0.006773
107,x15_B,0.011585
108,x15_C,0.009878
109,x15_D,0.005345
